<a href="https://colab.research.google.com/github/Gayeon6423/BusinessAI-Capston/blob/main/News_Data_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목차

### 1. Import library

### 2. Load Data

### 3. Filtering Data
- [3.1 Filtering : Kospi](#3.1-Sector-Filtering-:-Kospi)
- [3.2 Filtering : Kosdaq](#3.2-Sector-Filtering-:-Kosdaq)

## 1. Import library

In [3]:
# 데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
# 행렬 등 통계 연산 모듈
import numpy as np
from scipy.stats import iqr
from scipy.interpolate import interp1d
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
# 크롤링 및 스크래핑
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os
# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os
# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations
# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

Mounted at /content/drive


## 2. Load Data

In [7]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            sep = ",",
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  def loadTxTData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            sep = "|",
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  def loadExcelData(self, file_path: str, file_name_extension,
               columnTF: bool) -> pd.DataFrame():
    self.data = pd.read_excel(os.path.join(file_path + file_name_extension),
                              index_col = columnTF)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            sep = ",",
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    self.data = pd.concat(self.data)

    return self.data

In [8]:
mountInstance = LoadGoogleDriveData()

In [5]:
# Define the start and end dates
start_date = '2022-01'
end_date = '2023-07'

# Convert start and end dates to datetime objects
start = pd.to_datetime(start_date)
end = pd.to_datetime(end_date)

# Iterate over the date range
for date in pd.date_range(start, end, freq='M'):
    # Generate the file name for the specific month
    file_name = f"경제면_금융섹터_기사({date.strftime('%Y%m')})"
    print(file_name)

경제면_금융섹터_기사(202201)
경제면_금융섹터_기사(202202)
경제면_금융섹터_기사(202203)
경제면_금융섹터_기사(202204)
경제면_금융섹터_기사(202205)
경제면_금융섹터_기사(202206)
경제면_금융섹터_기사(202207)
경제면_금융섹터_기사(202208)
경제면_금융섹터_기사(202209)
경제면_금융섹터_기사(202210)
경제면_금융섹터_기사(202211)
경제면_금융섹터_기사(202212)
경제면_금융섹터_기사(202301)
경제면_금융섹터_기사(202302)
경제면_금융섹터_기사(202303)
경제면_금융섹터_기사(202304)
경제면_금융섹터_기사(202305)
경제면_금융섹터_기사(202306)


## 3. Filtering Data

### 3.1 Filtering : Kospi

In [25]:
# Iterate over the date range
for date in pd.date_range(start, end, freq='M'):

    # Read the CSV file
    news_data = mountInstance.loadData(
        file_path = '/content/drive/MyDrive/산업 AI 캡스톤/DATA/Original_News_Data/',
        file_name_extension = f"경제면_금융섹터_기사({date.strftime('%Y%m')}).csv",
        unicode = 'utf-8-sig', columnTF = False)

    # Drop null values
    news_data = news_data.dropna()

    # Filtering news : 코스피,코스피지수,KOSPI,kospi,Kospi,코스피200,Kospi지수,KOSPI지수,KOSPI200,kospi200 행 필터링
    news_data_kospi = news_data[news_data['content'].str.contains('코스피|코스피지수|KOSPI|kospi|Kospi|코스피200|Kospi지수|KOSPI지수|KOSPI200|kospi200')]

    # Generate the file name
    kospi_file_name = f"/content/drive/MyDrive/산업 AI 캡스톤/DATA/Kospi_Filtering_News_Data/news_{date.strftime('%Y%m')}_kospi.csv"

    # Save the filtered data to a new CSV file
    news_data_kospi.to_csv(kospi_file_name, encoding='utf-8-sig', index=False)

    # Display the first few rows of the filtered data
    print(f"First few rows of {kospi_file_name}, " "Rows number : ", len(news_data_kospi) )
    print()

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kospi_Filtering_News_Data/news_202201_kospi.csv, Rows number :  606

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kospi_Filtering_News_Data/news_202202_kospi.csv, Rows number :  387

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kospi_Filtering_News_Data/news_202203_kospi.csv, Rows number :  397

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kospi_Filtering_News_Data/news_202204_kospi.csv, Rows number :  328

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kospi_Filtering_News_Data/news_202205_kospi.csv, Rows number :  424

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kospi_Filtering_News_Data/news_202206_kospi.csv, Rows number :  593

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kospi_Filtering_News_Data/news_202207_kospi.csv, Rows number :  563

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DA

### 3.2 Filtering : Kosdaq

In [26]:
# Iterate over the date range
for date in pd.date_range(start, end, freq='M'):

    # Read the CSV file
    news_data = mountInstance.loadData(
        file_path = '/content/drive/MyDrive/산업 AI 캡스톤/DATA/Original_News_Data/',
        file_name_extension = f"경제면_금융섹터_기사({date.strftime('%Y%m')}).csv",
        unicode = 'utf-8-sig', columnTF = False)

    # Drop null values
    news_data = news_data.dropna()

    # Filtering news : 코스닥,코스닥지수,KOSDAP,kosdaq,Kosdaq,Kosdaq지수,KOSDAQ지수 행 필터링
    news_data_kosdaq = news_data[news_data['content'].str.contains('코스닥|코스닥지수|KOSDAP|kosdaq|Kosdaq|Kosdaq지수|KOSDAQ지수')]

    # Generate the file name
    kosdaq_file_name = f"/content/drive/MyDrive/산업 AI 캡스톤/DATA/Kosdaq_Filtering_News_Data/news_{date.strftime('%Y%m')}_kospi.csv"

    # Save the filtered data to a new CSV file
    news_data_kosdaq.to_csv(kosdaq_file_name, encoding='utf-8-sig', index=False)

    # Display the first few rows of the filtered data
    print(f"First few rows of {kosdaq_file_name}, " "Rows number : ", len(news_data_kosdaq) )
    print()

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kosdaq_Filtering_News_Data/news_202201_kospi.csv, Rows number :  510

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kosdaq_Filtering_News_Data/news_202202_kospi.csv, Rows number :  266

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kosdaq_Filtering_News_Data/news_202203_kospi.csv, Rows number :  274

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kosdaq_Filtering_News_Data/news_202204_kospi.csv, Rows number :  248

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kosdaq_Filtering_News_Data/news_202205_kospi.csv, Rows number :  274

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kosdaq_Filtering_News_Data/news_202206_kospi.csv, Rows number :  373

First few rows of /content/drive/MyDrive/산업 AI 캡스톤/DATA/Kosdaq_Filtering_News_Data/news_202207_kospi.csv, Rows number :  344

First few rows of /content/drive/MyDrive/산업 AI 캡ᄉ